In [84]:
from langchain_groq import ChatGroq 
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from datetime import datetime
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.agent_toolkits.load_tools import load_tools
load_dotenv()
import os 
api_key = os.getenv("GROQ_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [86]:
model = ChatGroq(
    api_key=str(api_key),
    model="openai/gpt-oss-20b"
)

In [87]:
structerd_ouput = model.with_structured_output(WearableSentiment)

In [88]:
query_prompt_template = PromptTemplate.from_template("""
You are a fitness analyst.  
Given wearable data:  
oxygen: {oxygen}  
sleep_duration: {sleep_duration}  
heart_rate: {heart_rate}  
steps: {steps}  
activity: {activity}  
gender: {gender}  
age: {age}  

Generate ONE concise Google search query  
that would help find useful, science-based insights or advice for the current health state.  
Only output the query text.
""")

wearable_data = {
    "oxygen": 95,
    "sleep_duration": "6h",
    "heart_rate": 88,
    "steps": 3200,
    "activity": ["Cardio" , "Gym"],
    "gender": "male",
    "age": 34
}


query_prompt = query_prompt_template.format(**wearable_data)
search_query = model.invoke(query_prompt)


In [89]:
print(f"Generated Search Query: {search_query.content}")
search_query = search_query.content

Generated Search Query: science‑based ways to increase sleep duration in an active 34‑year‑old male




In [90]:
search_tool = SerpAPIWrapper() 
search_results = search_tool.run(search_query)

In [ ]:
search_results

'['

In [91]:
analysis_prompt_template = PromptTemplate.from_template("""
You are a fitness analyser.
You have wearable readings:
oxygen: {oxygen}
sleep_duration: {sleep_duration}
heart_rate: {heart_rate}
steps: {steps}
activity: {activity}
gender: {gender}
age: {age}

Also, here are Google search results for relevant health advice:
{search_results}

Using BOTH the wearable readings and search results, return a JSON object that matches this schema exactly:
{{
  "oxygen": float,
  "sleep_duration": float,
  "heart_rate": float,
  "steps": int,
  "activity": ["string", "string"],
  "sentiment": ["string"],
  "insight": "string"
}}

Rules:
- activity must be a JSON array, e.g., ["walking", "yoga"]
- sentiment must be a JSON array, e.g., ["neutral"] or ["positive", "motivated"]
- insight must be a plain string
""")


In [98]:
analysis_prompt = analysis_prompt_template.format(
    **wearable_data,
    search_results=search_results
)

# structured_llm = model.with_structured_output(WearableSentiment)
final_analysis = model.invoke(analysis_prompt)


In [99]:
search_results

'[\'Making changes to your diet and lifestyle may help improve your sleep quality. Certain supplements and natural remedies could also be beneficial.\', \'-Avoid caffeine consumption for 10 hours before bedtime. -Do not drink alcohol or eat any food for 3 hours before going to bed. -Finish all work ...\', \'Regular physical activity can lead to improved sleep quality, reduced sleep latency, and better overall sleep quality.\', \'Based on available studies, “We have solid evidence that exercise does, in fact, help you fall asleep more quickly and improves sleep quality,” says Charlene ...\', \'The objective of this narrative review paper is to discuss about sleep duration needed across the lifespan. Sleep duration varies widely ...\', \'Be consistent: Going to bed and waking up at the same times every day helps the body develop a rhythm, which may make it easier to fall asleep ...\', \'Sleep Hygiene Tips for Athletes · Creating an appropriate sleep environment. · Avoid alcohol and caffe

In [101]:
print(final_analysis.content)

{
  "oxygen": 95.0,
  "sleep_duration": 6.0,
  "heart_rate": 88.0,
  "steps": 3200,
  "activity": ["Cardio", "Gym"],
  "sentiment": ["concerned"],
  "insight": "Your oxygen saturation is within normal limits, but your sleep duration of 6 hours falls below the 7‑9 hour range recommended for adults, which may increase risk of mental and physical health issues. Your resting heart rate of 88 bpm is slightly above average for a 34‑year‑old male, and your daily step count is low compared to typical activity levels. To improve overall well‑being, aim to increase nightly sleep to at least 7–9 hours, maintain a consistent bedtime routine, limit caffeine and alcohol intake before bed, and incorporate regular aerobic exercise. These changes are supported by evidence that both adequate sleep and consistent physical activity enhance sleep quality and cardiovascular health."
}
